# Factor Strategy

In [28]:
from class_port.port_factor import PortFactor
from class_model.model_prep import ModelPrep
from class_strat.strat import Strategy
from core.operation import *

class StratPortIVMD(Strategy):
    def __init__(self,
                 allocate=None,
                 current_date=None,
                 start_date=None,
                 threshold=None,
                 num_stocks=None,
                 window_port=None):

        '''
        allocate (float): Percentage of capital to allocate for this strategy
        current_date (str: YYYY-MM-DD): Current date (this will be used as the end date for backtest period)
        start_date (str: YYYY-MM-DD): Start date for backtest period
        num_stocks (int): Number of stocks to long/short
        threshold (int): Market cap threshold to determine if a stock is buyable/shortable
        window_port (int): Rolling window size to calculate inverse volatility
        '''

        super().__init__(allocate, current_date, threshold)
        self.allocate = allocate
        self.current_date = current_date
        self.start_date = start_date
        self.threshold = threshold
        self.num_stocks = num_stocks
        self.window_port = window_port

    def exec_backtest(self):
        print("-----------------------------------------------------------------BACKTEST PORT IVMD-------------------------------------------------------------------------------------")
        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # -----------------------------------------------------------------------------DATA--------------------------------------------------------------------------------------------
        live = True
        stock = read_stock(get_large(live) / 'permno_live.csv')

        # Load in datasets
        historical_price = pd.read_parquet(get_parquet(live) / 'data_price.parquet.brotli')
        fund_q = pd.read_parquet(get_parquet(live) / 'data_fund_raw_q.parquet.brotli', columns=['ltq', 'ceqq', 'prccq', 'cshoq', 'niq', 'dpq', 'xintq'])
        market = pd.read_parquet(get_parquet(live) / 'data_misc.parquet.brotli', columns=['market_cap'])

        historical_price = set_timeframe(historical_price, self.start_date, self.current_date)
        # Create returns and resample fund_q date index to daily
        ret_price = create_return(historical_price, [1])
        # ret_price = ret_price.groupby('permno').shift(-2)
        date_index = historical_price.drop(historical_price.columns, axis=1)
        # fund_q = fund_q.groupby('permno').shift(3)
        fund_q = date_index.merge(fund_q, left_index=True, right_index=True, how='left').groupby('permno').ffill()

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # ------------------------------------------------------------------------LOAD FACTOR DATA-------------------------------------------------------------------------------------
        print("-------------------------------------------------------------------LOAD FACTOR DATA-------------------------------------------------------------------------------------")
        # Fundamental
        accrual = ModelPrep(live=live, factor_name='factor_accrual', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        # accrual = accrual.groupby('permno').shift(3)
        comp_debt = ModelPrep(live=live, factor_name='factor_comp_debt', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        # comp_debt = comp_debt.groupby('permno').shift(3)
        inv_growth = ModelPrep(live=live, factor_name='factor_inv_growth', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        # inv_growth = inv_growth.groupby('permno').shift(3)
        pcttoacc = ModelPrep(live=live, factor_name='factor_pcttotacc', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        # pcttoacc = pcttoacc.groupby('permno').shift(3)
        chtax = ModelPrep(live=live, factor_name='factor_chtax', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        # chtax = chtax.groupby('permno').shift(3)
        net_debt_finance = ModelPrep(live=live, factor_name='factor_net_debt_finance', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        # net_debt_finance = net_debt_finance.groupby('permno').shift(3)
        noa = ModelPrep(live=live, factor_name='factor_noa', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        # noa = noa.groupby('permno').shift(3)
        invest_ppe = ModelPrep(live=live, factor_name='factor_invest_ppe_inv', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        # invest_ppe = invest_ppe.groupby('permno').shift(3)
        cheq = ModelPrep(live=live, factor_name='factor_cheq', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        # cheq = cheq.groupby('permno').shift(3)
        xfin = ModelPrep(live=live, factor_name='factor_xfin', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        # xfin = xfin.groupby('permno').shift(3)
        emmult = ModelPrep(live=live, factor_name='factor_emmult', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        # emmult = emmult.groupby('permno').shift(3)
        grcapx = ModelPrep(live=live, factor_name='factor_grcapx', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        # grcapx = grcapx.groupby('permno').shift(3)
        fund_q['ev_to_ebitda'] = (fund_q['ltq'] + fund_q['ceqq'] + (fund_q['prccq'] * fund_q['cshoq'])) / (fund_q['niq'] + fund_q['dpq'] + fund_q['xintq'])
        fund_q = fund_q.replace([np.inf, -np.inf], np.nan)
        fund_factor = fund_q[['ev_to_ebitda']]

        # Momentum
        mom_season = ModelPrep(live=live, factor_name='factor_mom_season', group='permno', interval='D', kind='mom', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        mom_season6 = ModelPrep(live=live, factor_name='factor_mom_season6', group='permno', interval='D', kind='mom', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        load_ret = ModelPrep(live=live, factor_name='factor_load_ret', group='permno', interval='D', kind='loading', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        mom_season_short = ModelPrep(live=live, factor_name='factor_mom_season_short', group='permno', interval='D', kind='mom', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()

        # Defensive
        sb_sector = ModelPrep(live=live, factor_name='factor_sb_sector', group='permno', interval='D', kind='price', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()
        sb_pca = ModelPrep(live=live, factor_name='factor_sb_pca', group='permno', interval='D', kind='price', stock=stock, div=False, start=self.start_date, end=self.current_date, save=False).prep()

        # Merge into one dataframe
        factor_data = (pd.merge(ret_price, sb_sector, left_index=True, right_index=True, how='left')
                       .merge(sb_pca, left_index=True, right_index=True, how='left')
                       .merge(accrual, left_index=True, right_index=True, how='left')
                       .merge(comp_debt, left_index=True, right_index=True, how='left')
                       .merge(inv_growth, left_index=True, right_index=True, how='left')
                       .merge(pcttoacc, left_index=True, right_index=True, how='left')
                       .merge(mom_season, left_index=True, right_index=True, how='left')
                       .merge(mom_season6, left_index=True, right_index=True, how='left')
                       .merge(chtax, left_index=True, right_index=True, how='left')
                       .merge(net_debt_finance, left_index=True, right_index=True, how='left')
                       .merge(noa, left_index=True, right_index=True, how='left')
                       .merge(invest_ppe, left_index=True, right_index=True, how='left')
                       .merge(cheq, left_index=True, right_index=True, how='left')
                       .merge(xfin, left_index=True, right_index=True, how='left')
                       .merge(emmult, left_index=True, right_index=True, how='left')
                       .merge(grcapx, left_index=True, right_index=True, how='left')
                       .merge(mom_season_short, left_index=True, right_index=True, how='left')
                       .merge(load_ret, left_index=True, right_index=True, how='left')
                       .merge(fund_factor, left_index=True, right_index=True, how='left')
                       .merge(market, left_index=True, right_index=True, how='left'))

        factor_data['accruals'] = factor_data.groupby('permno')['accruals'].ffill()
        factor_data['comp_debt_iss'] = factor_data.groupby('permno')['comp_debt_iss'].ffill()
        factor_data['inv_growth'] = factor_data.groupby('permno')['inv_growth'].ffill()
        factor_data['chtax'] = factor_data.groupby('permno')['chtax'].ffill()
        factor_data['net_debt_fin'] = factor_data.groupby('permno')['net_debt_fin'].ffill()
        factor_data['noa'] = factor_data.groupby('permno')['noa'].ffill()
        factor_data['invest_ppe_inv'] = factor_data.groupby('permno')['invest_ppe_inv'].ffill()
        factor_data['cheq'] = factor_data.groupby('permno')['cheq'].ffill()
        factor_data['pct_tot_acc'] = factor_data.groupby('permno')['pct_tot_acc'].ffill()
        factor_data['xfin'] = factor_data.groupby('permno')['xfin'].ffill()
        factor_data['emmult'] = factor_data.groupby('permno')['emmult'].ffill()
        factor_data['grcapx'] = factor_data.groupby('permno')['grcapx'].ffill()

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # ----------------------------------------------------------------------------GET RANKINGS-------------------------------------------------------------------------------------
        print("-----------------------------------------------------------------------GET RANKINGS-------------------------------------------------------------------------------------")
        factors = [
            "XLB_RET_01_sector_01_126",
            "XLE_RET_01_sector_01_126",
            "XLF_RET_01_sector_01_126",
            "XLI_RET_01_sector_01_126",
            "XLK_RET_01_sector_01_126",
            "XLP_RET_01_sector_01_126",
            "XLU_RET_01_sector_01_126",
            "XLV_RET_01_sector_01_126",
            "XLY_RET_01_sector_01_126",
            'PCA_Return_1_ret_pca_01_126',
            'PCA_Return_2_ret_pca_01_126',
            'PCA_Return_3_ret_pca_01_126',
            'PCA_Return_4_ret_pca_01_126',
            'PCA_Return_5_ret_pca_01_126',
            "accruals",
            "inv_growth",
            "comp_debt_iss",
            "pct_tot_acc",
            'chtax',
            'net_debt_fin',
            'noa',
            'invest_ppe_inv',
            'cheq',
            'xfin',
            'emmult',
            'grcapx',
            'ev_to_ebitda',
            'load_ret_1',
            'load_ret_2',
            'load_ret_3',
            'load_ret_4',
            'load_ret_5',
            "mom_season",
            "mom_season_short",
            "mom_season_6"
        ]

        filname = f"port_ivmd_{date.today().strftime('%Y%m%d')}.html"
        dir_path = get_strat_port_ivmd() / 'report' / filname

        long_short_stocks = PortFactor(data=factor_data, window=self.window_port, num_stocks=self.num_stocks, factors=factors,
                                       threshold=self.threshold, backtest=False, dir_path=dir_path).create_factor_port()

        return long_short_stocks

    def exec_live(self):
        print("-------------------------------------------------------------------EXEC PORT IVMD--------------------------------------------------------------------------------------")
        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # -----------------------------------------------------------------------------DATA--------------------------------------------------------------------------------------------
        live = True
        stock = read_stock(get_large(live) / 'permno_live.csv')
        window_date = (pd.to_datetime(self.current_date) - BDay(252)).strftime('%Y-%m-%d')
        
        # Load in datasets
        fund_q = pd.read_parquet(get_parquet(live) / 'data_fund_raw_q.parquet.brotli', columns=['ltq', 'ceqq', 'prccq', 'cshoq', 'niq', 'dpq', 'xintq'])
        market = pd.read_parquet(get_parquet(live) / 'data_misc.parquet.brotli', columns=['market_cap'])
        price = pd.read_parquet(get_parquet(live) / 'data_price.parquet.brotli')
        price = set_timeframe(price, self.start_date, self.current_date)

        # Create returns crop into window data
        ret_price = create_return(price, [1])
        ret_price = window_data(data=ret_price, date=self.current_date, window=self.window_port * 2)

        # Resample fund_q date index to daily and crop into window data
        year_price = window_data(data=price, date=self.current_date, window=252 * 2)
        date_index = year_price.drop(year_price.columns, axis=1)
        fund_q = date_index.merge(fund_q, left_index=True, right_index=True, how='left').groupby('permno').ffill()
        fund_q = window_data(data=fund_q, date=self.current_date, window=self.window_port * 2)

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # ------------------------------------------------------------------------LOAD FACTOR DATA-------------------------------------------------------------------------------------
        print("-------------------------------------------------------------------LOAD FACTOR DATA-------------------------------------------------------------------------------------")
        # Fundamental
        accrual = ModelPrep(live=live, factor_name='factor_accrual', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        comp_debt = ModelPrep(live=live, factor_name='factor_comp_debt', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        inv_growth = ModelPrep(live=live, factor_name='factor_inv_growth', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        pcttoacc = ModelPrep(live=live, factor_name='factor_pcttotacc', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        chtax = ModelPrep(live=live, factor_name='factor_chtax', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        net_debt_finance = ModelPrep(live=live, factor_name='factor_net_debt_finance', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        noa = ModelPrep(live=live, factor_name='factor_noa', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        invest_ppe = ModelPrep(live=live, factor_name='factor_invest_ppe_inv', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        cheq = ModelPrep(live=live, factor_name='factor_cheq', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        xfin = ModelPrep(live=live, factor_name='factor_xfin', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        emmult = ModelPrep(live=live, factor_name='factor_emmult', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        grcapx = ModelPrep(live=live, factor_name='factor_grcapx', group='permno', interval='M', kind='fundamental', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        fund_q['ev_to_ebitda'] = (fund_q['ltq'] + fund_q['ceqq'] + (fund_q['prccq'] * fund_q['cshoq'])) / (fund_q['niq'] + fund_q['dpq'] + fund_q['xintq'])
        fund_q = fund_q.replace([np.inf, -np.inf], np.nan)
        fund_factor = fund_q[['ev_to_ebitda']]

        # Momentum
        mom_season = ModelPrep(live=live, factor_name='factor_mom_season', group='permno', interval='D', kind='mom', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        mom_season6 = ModelPrep(live=live, factor_name='factor_mom_season6', group='permno', interval='D', kind='mom', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        load_ret = ModelPrep(live=live, factor_name='factor_load_ret', group='permno', interval='D', kind='loading', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        mom_season_short = ModelPrep(live=live, factor_name='factor_mom_season_short', group='permno', interval='D', kind='mom', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()

        # Defensive
        sb_sector = ModelPrep(live=live, factor_name='factor_sb_sector', group='permno', interval='D', kind='price', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()
        sb_pca = ModelPrep(live=live, factor_name='factor_sb_pca', group='permno', interval='D', kind='price', stock=stock, div=False, start=window_date, end=self.current_date, save=False).prep()

        # Merge into one dataframe
        factor_data = (pd.merge(ret_price, sb_sector, left_index=True, right_index=True, how='left')
                          .merge(sb_pca, left_index=True, right_index=True, how='left')
                          .merge(accrual, left_index=True, right_index=True, how='left')
                          .merge(comp_debt, left_index=True, right_index=True, how='left')
                          .merge(inv_growth, left_index=True, right_index=True, how='left')
                          .merge(pcttoacc, left_index=True, right_index=True, how='left')
                          .merge(mom_season, left_index=True, right_index=True, how='left')
                          .merge(mom_season6, left_index=True, right_index=True, how='left')
                          .merge(chtax, left_index=True, right_index=True, how='left')
                          .merge(net_debt_finance, left_index=True, right_index=True, how='left')
                          .merge(noa, left_index=True, right_index=True, how='left')
                          .merge(invest_ppe, left_index=True, right_index=True, how='left')
                          .merge(cheq, left_index=True, right_index=True, how='left')
                          .merge(xfin, left_index=True, right_index=True, how='left')
                          .merge(emmult, left_index=True, right_index=True, how='left')
                          .merge(grcapx, left_index=True, right_index=True, how='left')
                          .merge(mom_season_short, left_index=True, right_index=True, how='left')
                          .merge(load_ret, left_index=True, right_index=True, how='left')
                          .merge(fund_factor, left_index=True, right_index=True, how='left')
                          .merge(market, left_index=True, right_index=True, how='left'))

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # ----------------------------------------------------------------------------GET RANKINGS-------------------------------------------------------------------------------------
        print("-----------------------------------------------------------------------GET RANKINGS-------------------------------------------------------------------------------------")
        factors = [
            "XLB_RET_01_sector_01_126",
            "XLE_RET_01_sector_01_126",
            "XLF_RET_01_sector_01_126",
            "XLI_RET_01_sector_01_126",
            "XLK_RET_01_sector_01_126",
            "XLP_RET_01_sector_01_126",
            "XLU_RET_01_sector_01_126",
            "XLV_RET_01_sector_01_126",
            "XLY_RET_01_sector_01_126",
            'PCA_Return_1_ret_pca_01_126',
            'PCA_Return_2_ret_pca_01_126',
            'PCA_Return_3_ret_pca_01_126',
            'PCA_Return_4_ret_pca_01_126',
            'PCA_Return_5_ret_pca_01_126',
            "accruals",
            "inv_growth",
            "comp_debt_iss",
            "pct_tot_acc",
            'chtax',
            'net_debt_fin',
            'noa',
            'invest_ppe_inv',
            'cheq',
            'xfin',
            'emmult',
            'grcapx',
            'ev_to_ebitda',
            'load_ret_1',
            'load_ret_2',
            'load_ret_3',
            'load_ret_4',
            'load_ret_5',
            "mom_season",
            "mom_season_short",
            "mom_season_6"
        ]

        # Forward Fill Factors
        factor_data[factors] = factor_data.groupby('permno')[factors].ffill()

        filname = f"port_ivmd_{date.today().strftime('%Y%m%d')}"
        dir_path = get_strat_port_ivmd() / 'report' / filname

        latest_window_data = window_data(data=factor_data, date=self.current_date, window=self.window_port*2)
        long_short_stocks = PortFactor(data=latest_window_data, window=self.window_port, num_stocks=self.num_stocks, factors=factors,
                                       threshold=self.threshold, backtest=False, dir_path=dir_path).create_factor_port()
        return long_short_stocks

--

In [29]:
# Initalize Factor Strategy
strat_crit = json.load(open(get_config() / 'strat_crit.json'))
current_date = '2019-12-31'
strat_port_ivmd = StratPortIVMD(allocate=strat_crit['port_ivmd']['allocate'], current_date=current_date, start_date=strat_crit['port_ivmd']['start_backtest'], threshold=strat_crit['port_ivmd']['threshold'], num_stocks=strat_crit['port_ivmd']['per_side'][0], window_port=21)

\\

In [30]:
# Get Backtest Weights
factor_backtest = strat_port_ivmd.exec_backtest()

-----------------------------------------------------------------BACKTEST PORT IVMD-------------------------------------------------------------------------------------
|\-------------------------------------------------------------------LOAD FACTOR DATA-------------------------------------------------------------------------------------
Creating factor_accrual ------------------------------------ | ✔
Shape: (2835174, 1)
Creating factor_comp_debt ---------------------------------- |-✔
Shape: (2835174, 1)
Creating factor_inv_growth ---------------------------------| ✔
/Shape: (2835174, 1)
Creating factor_pcttotacc ---------------------------------- | ✔
Shape: (2835174, 1)
Creating factor_chtax --------------------------------------| ✔
\Shape: (2835174, 1)
Creating factor_net_debt_finance --------------------------- | ✔
Shape: (2835174, 1)
Creating factor_noa ---------------------------------------- | ✔
Shape: (2835174, 1)
Creating factor_invest_ppe_inv ----------------------------- | ✔


In [31]:
# Get Livetest Weights
factor_livetest = strat_port_ivmd.exec_live()

-------------------------------------------------------------------EXEC PORT IVMD--------------------------------------------------------------------------------------
---------------------------------------------------------------------LOAD FACTOR DATA-------------------------------------------------------------------------------------
Creating factor_accrual ------------------------------------|| ✔
Shape: (197932, 1)
Creating factor_comp_debt -----------------------------------| ✔
Shape: (197932, 1)
Creating factor_inv_growth ---------------------------------|| ✔
Shape: (197932, 1)
Creating factor_pcttotacc -----------------------------------| ✔
Shape: (197932, 1)
Creating factor_chtax --------------------------------------|| ✔
Shape: (197932, 1)
Creating factor_net_debt_finance ----------------------------| ✔
Shape: (197932, 1)
Creating factor_noa ----------------------------------------\| ✔
Shape: (197932, 1)
Creating factor_invest_ppe_inv ----------------------------- | ✔
Shape: (

In [35]:
factor_backtest.tail(25)

,,Open,High,Low,Close,Volume,RET_01,ALPHA_sector_01_21,XLB_RET_01_sector_01_21,XLE_RET_01_sector_01_21,XLF_RET_01_sector_01_21,...,mom_season_Rank,mom_season_short_Rank,mom_season_6_Rank,avg_rank,rank_weight,vol,inv_vol_weight,adj_weight,final_weight,total_ret
permno,date,,,,,,,,,,,,,,,,,,,,,
16656,2019-12-31,3.5900,3.6400,3.69000,184.500000,247369.0,0.045326,-0.015894,-1.063082,-1.595991,-23.813438,...,7.0,1.0,169.0,316.275862,9.036453,0.261173,3.828881,34.599502,-0.000543,-0.000025
15950,2019-12-31,2.1100,2.2700,2.28000,319.193616,4291468.0,0.070423,-0.008038,6.075133,0.219154,3.065406,...,3.0,18.0,317.0,320.545455,9.158442,0.212470,4.706550,43.104667,-0.000676,-0.000048
18089,2019-12-31,1.2600,1.1300,1.14000,1412.500000,96850.0,-0.083387,0.003020,-1.503527,8.019351,6.187162,...,226.0,720.0,2.0,295.121212,8.432035,0.113222,8.832229,74.473663,-0.001169,0.000097
17273,2019-12-31,1.0500,1.0400,1.05000,87.211740,829939.0,0.000000,0.004202,-2.644982,6.399078,-0.031825,...,705.0,4.0,274.0,354.625000,10.132143,0.073191,13.662816,138.433606,-0.002172,-0.000000
90494,2019-12-31,1.6500,1.6800,1.69000,84.500000,785927.0,0.011976,0.019431,1.063414,-1.026411,-0.565368,...,22.0,691.0,103.0,263.852941,7.538655,0.051002,19.607150,147.811546,-0.002319,-0.000028
13688,2019-12-31,10.6700,10.8600,10.87000,10.870000,10593479.0,0.006481,0.056465,-3.833518,-2.679264,4.290089,...,5.0,595.0,704.0,367.484848,10.499567,0.068889,14.516151,152.413303,-0.002391,-0.000016
15656,2019-12-31,1.3400,1.3100,1.32000,330.000000,1590003.0,0.000000,-0.010483,-2.314027,-0.411249,-0.727254,...,515.0,29.0,617.0,322.656250,9.218750,0.059190,16.894789,155.748838,-0.002444,-0.000000
39490,2019-12-31,25.2300,25.5800,25.59000,25.590000,3349386.0,0.003529,0.014044,-1.456199,2.822368,-4.780299,...,610.0,712.0,69.0,311.485714,8.899592,0.054541,18.334973,163.173780,-0.002560,-0.000009
89911,2019-12-31,0.7290,0.7399,0.74000,7.362000,2158315.0,0.018257,0.025030,-0.220949,-7.363423,12.953880,...,69.0,367.0,681.0,347.314286,9.923265,0.060130,16.630706,165.030908,-0.002589,-0.000047


\\

In [36]:
factor_livetest.tail(25)

,,Open,High,Low,Close,Volume,RET_01,ALPHA_sector_01_21,XLB_RET_01_sector_01_21,XLE_RET_01_sector_01_21,XLF_RET_01_sector_01_21,...,mom_season_Rank,mom_season_short_Rank,mom_season_6_Rank,avg_rank,rank_weight,vol,inv_vol_weight,adj_weight,final_weight,total_ret
permno,date,,,,,,,,,,,,,,,,,,,,,
16656,2019-12-31,3.5900,3.6400,3.69000,184.500000,247369.0,0.045326,-0.015894,-1.063082,-1.595991,-23.813438,...,7.0,1.0,169.0,316.275862,9.036453,0.261173,3.828881,34.599502,-0.000544,-0.000025
15950,2019-12-31,2.1100,2.2700,2.28000,319.193616,4291468.0,0.070423,-0.008038,6.075133,0.219154,3.065406,...,3.0,18.0,317.0,319.272727,9.122078,0.212470,4.706550,42.933520,-0.000675,-0.000048
18089,2019-12-31,1.2600,1.1300,1.14000,1412.500000,96850.0,-0.083387,0.003020,-1.503527,8.019351,6.187162,...,226.0,720.0,2.0,286.750000,8.192857,0.113222,8.832229,72.361192,-0.001137,0.000095
17273,2019-12-31,1.0500,1.0400,1.05000,87.211740,829939.0,0.000000,0.004202,-2.644982,6.399078,-0.031825,...,705.0,4.0,274.0,349.483871,9.985253,0.073191,13.662816,136.426683,-0.002144,-0.000000
90494,2019-12-31,1.6500,1.6800,1.69000,84.500000,785927.0,0.011976,0.019431,1.063414,-1.026411,-0.565368,...,22.0,691.0,103.0,262.735294,7.506723,0.051002,19.607150,147.185435,-0.002313,-0.000028
13688,2019-12-31,10.6700,10.8600,10.87000,10.870000,10593479.0,0.006481,0.056465,-3.833518,-2.679264,4.290089,...,5.0,595.0,704.0,365.878788,10.453680,0.068889,14.516151,151.747194,-0.002385,-0.000015
15656,2019-12-31,1.3400,1.3100,1.32000,330.000000,1590003.0,0.000000,-0.010483,-2.314027,-0.411249,-0.727254,...,515.0,29.0,617.0,321.437500,9.183929,0.059190,16.894789,155.160537,-0.002439,-0.000000
39490,2019-12-31,25.2300,25.5800,25.59000,25.590000,3349386.0,0.003529,0.014044,-1.456199,2.822368,-4.780299,...,610.0,712.0,69.0,310.428571,8.869388,0.054541,18.334973,162.619989,-0.002556,-0.000009
89911,2019-12-31,0.7290,0.7399,0.74000,7.362000,2158315.0,0.018257,0.025030,-0.220949,-7.363423,12.953880,...,69.0,367.0,681.0,346.457143,9.898776,0.060130,16.630706,164.623625,-0.002587,-0.000047


-/

# Factor Mrev

In [92]:
import quantstats as qs

from core.operation import *
from class_mrev.mrev_sd_epsil import MrevSDEpsil
from class_strat.strat import Strategy

class StratMrevETF(Strategy):
    def __init__(self,
                 allocate=None,
                 current_date=None,
                 start_date=None,
                 threshold=None,
                 window_epsil=None,
                 sbo=None,
                 sso=None,
                 sbc=None,
                 ssc=None):

        '''
        allocate (float): Percentage of capital to allocate for this strategy
        current_date (str: YYYY-MM-DD): Current date (this will be used as the end date for backtest period)
        start_date (str: YYYY-MM-DD): Start date for backtest period
        threshold (int): Market cap threshold to determine if a stock is buyable/shortable
        window_epsil (int): Rolling window size to calculate standardized s-score
        sbo (float): Threshold to determine buy signal
        sso (float): Threshold to determine sell signal
        sbc (float): Threshold to determine close buy signal
        ssc (float): Threshold to determine close sell signal
        '''

        super().__init__(allocate, current_date, threshold)
        self.allocate = allocate
        self.current_date = current_date
        self.start_date = start_date
        self.threshold = threshold
        self.window_epsil = window_epsil
        self.sbo = sbo
        self.sso = sso
        self.sbc = sbc
        self.ssc = ssc

    def exec_backtest(self):
        print("-----------------------------------------------------------------BACKTEST MREV ETF--------------------------------------------------------------------------------------")
        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # -----------------------------------------------------------------------------DATA--------------------------------------------------------------------------------------------
        # Create MrevSDEpsil Class
        mrev_sd_epsil = MrevSDEpsil(name='sector', threshold=self.threshold, sbo=self.sbo, sso=self.sso, sbc=self.sbc, ssc=self.ssc)

        # Params
        live = True
        hedge_ticker = ['XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLK', 'XLU']

        # Load in hedge dataset
        hedge_ret = get_data_fmp(ticker_list=hedge_ticker, start=self.start_date, current_date=self.current_date)
        hedge_ret = hedge_ret[['Open', 'High', 'Low', 'Volume', 'Adj Close']]
        hedge_ret = hedge_ret.rename(columns={'Adj Close': 'Close'})
        hedge_ret = hedge_ret.loc[~hedge_ret.index.duplicated(keep='first')]

        # Create returns and unstack dataframe to only have 'date' index and 'ticker' columns
        hedge_ret = create_return(hedge_ret, [1])
        hedge_ret = hedge_ret.drop(['Close', 'High', 'Low', 'Open', 'Volume'], axis=1)
        hedge_ret = hedge_ret.unstack('ticker').swaplevel(axis=1)
        hedge_ret.columns = ['_'.join(col).strip() for col in hedge_ret.columns.values]
        hedge_ret = hedge_ret.fillna(0)

        # Load in datasets
        stock = read_stock(get_large(live) / 'permno_live.csv')
        historical_price = pd.read_parquet(get_parquet(live) / 'data_price.parquet.brotli')
        historical_price = set_timeframe(historical_price, self.start_date, self.current_date)
        
        # Create returns
        price = create_return(historical_price, [1])
        price = price.fillna(0)

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # -----------------------------------------------------------------------PERFORM ROLLING OLS-----------------------------------------------------------------------------------
        print("------------------------------------------------------------------PERFORM ROLLING OLS-----------------------------------------------------------------------------------")
        # Params
        ret = 'RET_01'
        factor_col = hedge_ret.columns.tolist()
        name = 'sector_01'

        # Execute Rolling LR
        beta_data = rolling_ols_parallel(data=price, ret=ret, factor_data=hedge_ret, factor_cols=factor_col, window=self.window_epsil, name=name)

        # Retrieve required data
        beta_data = beta_data[beta_data.columns[1:len(factor_col) + 2]]
        beta_data = beta_data.fillna(0)

        # Calculate rolling mean, standard deviation, and z-score
        rolling_mean = beta_data.groupby('permno')[f'epsil_{name}_{self.window_epsil:02}'].rolling(window=self.window_epsil).mean().reset_index(level=0, drop=True)
        rolling_std = beta_data.groupby('permno')[f'epsil_{name}_{self.window_epsil:02}'].rolling(window=self.window_epsil).std().reset_index(level=0, drop=True)
        beta_data['s_score'] = (beta_data[f'epsil_{name}_{self.window_epsil:02}'] - rolling_mean) / rolling_std

        # Convert Hedge Dataframe to multi-index
        hedge_data = mrev_sd_epsil._create_multi_index(hedge_ret, stock)

        # Merge beta_data and hedge_multi
        comb_data = beta_data.merge(hedge_data, left_index=True, right_index=True, how='left')
        comb_data = comb_data.merge(price[['RET_01']], left_index=True, right_index=True, how='left')
        comb_data = comb_data.fillna(0)

        # Retrieve required data
        ret_columns = [col for col in comb_data.columns if "RET_01" in col]
        comb_data = comb_data[['s_score'] + ret_columns]

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # -------------------------------------------------------------------------CREATE SIGNALS--------------------------------------------------------------------------------------
        print("--------------------------------------------------------------------CREATE SIGNALS--------------------------------------------------------------------------------------")
        # Add Market Cap data
        market = pd.read_parquet(get_parquet(live) / 'data_misc.parquet.brotli', columns=['market_cap'])
        comb_data = comb_data.merge(market, left_index=True, right_index=True, how='left')

        # Create Signals
        signal_data = mrev_sd_epsil._create_signal(comb_data)

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # -------------------------------------------------------------------------BACKTEST STRATEGY-----------------------------------------------------------------------------------
        print("--------------------------------------------------------------------BACKTEST STRATEGY-----------------------------------------------------------------------------------")
        # Calculate total returns and weights
        total_ret, beta_weight, stock_weight = mrev_sd_epsil.calc_total_ret(signal_data, hedge_ret)
        
        return total_ret, beta_weight, stock_weight, comb_data

    def exec_live(self):
        print("-------------------------------------------------------------------EXEC MREV ETF----------------------------------------------------------------------------------------")
        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # -----------------------------------------------------------------------------DATA--------------------------------------------------------------------------------------------
        # Create MrevSDEpsil Class
        mrev_sd_epsil = MrevSDEpsil(name='sector', threshold=self.threshold, sbo=self.sbo, sso=self.sso, sbc=self.sbc, ssc=self.ssc)

        # Params
        live = True
        hedge_ticker = ['XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLK', 'XLU']

        # Load in datasets
        stock = read_stock(get_large(live) / 'permno_live.csv')
        price = pd.read_parquet(get_parquet(live) / 'data_price.parquet.brotli')
        price = set_timeframe(price, self.start_date, self.current_date)
        
        # Load in hedge dataset
        hedge_ret = get_data_fmp(ticker_list=hedge_ticker, start=self.start_date, current_date=self.current_date)
        hedge_ret = hedge_ret[['Open', 'High', 'Low', 'Volume', 'Adj Close']]
        hedge_ret = hedge_ret.rename(columns={'Adj Close': 'Close'})
        hedge_ret = hedge_ret.loc[~hedge_ret.index.duplicated(keep='first')]

        # Create returns and unstack dataframe to only have 'date' index and 'ticker' columns
        hedge_ret = create_return(hedge_ret, [1])
        hedge_ret = hedge_ret.drop(['Close', 'High', 'Low', 'Open', 'Volume'], axis=1)
        hedge_ret = hedge_ret.unstack('ticker').swaplevel(axis=1)
        hedge_ret.columns = ['_'.join(col).strip() for col in hedge_ret.columns.values]
        hedge_ret = hedge_ret.fillna(0)

        # Create returns
        price = create_return(price, [1])
        price = price.fillna(0)

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # -----------------------------------------------------------------------PERFORM ROLLING OLS-----------------------------------------------------------------------------------
        print("------------------------------------------------------------------PERFORM ROLLING OLS-----------------------------------------------------------------------------------")
        # Params
        ret = 'RET_01'
        factor_col = hedge_ret.columns.tolist()
        name = 'sector_01'

        # Execute Rolling LR
        window_price = window_data(data=price, date=self.current_date, window=self.window_epsil*3)
        beta_data = rolling_ols_parallel(data=window_price, ret=ret, factor_data=hedge_ret, factor_cols=factor_col, window=self.window_epsil, name=name)

        # Retrieve required data
        beta_data = beta_data[beta_data.columns[1:len(factor_col) + 2]]
        beta_data = beta_data.fillna(0)

        # Calculate rolling mean, standard deviation, and z-score
        rolling_mean = beta_data.groupby('permno')[f'epsil_{name}_{self.window_epsil:02}'].rolling(window=self.window_epsil).mean().reset_index(level=0, drop=True)
        rolling_std = beta_data.groupby('permno')[f'epsil_{name}_{self.window_epsil:02}'].rolling(window=self.window_epsil).std().reset_index(level=0, drop=True)
        beta_data['s_score'] = (beta_data[f'epsil_{name}_{self.window_epsil:02}'] - rolling_mean) / rolling_std

        # Convert Hedge Dataframe to multi-index
        hedge_data = mrev_sd_epsil._create_multi_index(hedge_ret, stock)

        # Merge beta_data and hedge_multi
        comb_data = beta_data.merge(hedge_data, left_index=True, right_index=True, how='left')
        comb_data = comb_data.merge(window_price[['RET_01']], left_index=True, right_index=True, how='left')
        comb_data = comb_data.fillna(0)

        # Retrieve required data
        ret_columns = [col for col in comb_data.columns if "RET_01" in col]
        comb_data = comb_data[['s_score'] + ret_columns]

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # -------------------------------------------------------------------------CREATE SIGNALS--------------------------------------------------------------------------------------
        print("--------------------------------------------------------------------CREATE SIGNALS--------------------------------------------------------------------------------------")
        # Add Market Cap data
        market = pd.read_parquet(get_parquet(live) / 'data_misc.parquet.brotli', columns=['market_cap'])
        comb_data = comb_data.merge(market, left_index=True, right_index=True, how='left')

        # Create Signals
        window_comb = window_data(data=comb_data, date=self.current_date, window=21*2)
        signal_data = mrev_sd_epsil._create_signal(window_comb)

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # ---------------------------------------------------------------------------GET STOCKS----------------------------------------------------------------------------------------
        print("----------------------------------------------------------------------GET STOCKS----------------------------------------------------------------------------------------")
        # Calculate total returns and weights
        total_ret, beta_weight, stock_weight = mrev_sd_epsil.calc_total_ret(signal_data, hedge_ret)
        return total_ret, beta_weight, stock_weight, comb_data

\

In [83]:
# Initalize Factor Strategy
strat_crit = json.load(open(get_config() / 'strat_crit.json'))
current_date = '2024-01-04'
strat_mrev_etf = StratMrevETF(allocate=strat_crit['mrev_etf']['allocate'], current_date=current_date, start_date=strat_crit['mrev_etf']['start_backtest'], threshold=strat_crit['mrev_etf']['threshold'], window_epsil=168, sbo=0.85, sso=0.85, sbc=0.25, ssc=0.25)

\

In [84]:
# Get Backtest Weights
mrev_backtest = strat_mrev_etf.exec_backtest()

-----------------------------------------------------------------BACKTEST MREV ETF--------------------------------------------------------------------------------------


Fetching data:   0%|                                                                                                                                                                                                                           | 0/9 [00:00<?, ?ticker/s]

|

Fetching data:  11%|███████████████████████▍                                                                                                                                                                                           | 1/9 [00:00<00:07,  1.07ticker/s]

/

...

Fetching data:  89%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                       | 8/9 [00:07<00:01,  1.02s/ticker]

-

Fetching data: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:08<00:00,  1.02ticker/s]

\

\------------------------------------------------------------------PERFORM ROLLING OLS-----------------------------------------------------------------------------------
-|--------------------------------------------------------------------CREATE SIGNALS-------------------------------------------------------------------------------------\
-/--------------------------------------------------------------------BACKTEST STRATEGY-----------------------------------------------------------------------------------
Get hedge weights...
Get net hedge betas...
\Normalize weights...
|Get net hedge returns...
Get total hedge return...
Get daily returns...
|Get total returns...
/

In [93]:
# Get Livetest Weights
mrev_livetest = strat_mrev_etf.exec_live()

-------------------------------------------------------------------EXEC MREV ETF----------------------------------------------------------------------------------------
-

Fetching data:   0%|                                                                                                                                                                    | 0/9 [00:00<?, ?ticker/s]

-

Fetching data:  11%|█████████████████▎                                                                                                                                          | 1/9 [00:00<00:07,  1.10ticker/s]

\

...

Fetching data:  89%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                 | 8/9 [00:07<00:01,  1.01s/ticker]

/

Fetching data: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:08<00:00,  1.01ticker/s]

-

\------------------------------------------------------------------PERFORM ROLLING OLS-----------------------------------------------------------------------------------
-|--------------------------------------------------------------------CREATE SIGNALS--------------------------------------------------------------------------------------
\/----------------------------------------------------------------------GET STOCKS----------------------------------------------------------------------------------------
Get hedge weights...
Get net hedge betas...
Normalize weights...
Get net hedge returns...
Get total hedge return...
Get daily returns...
Get total returns...
-

In [94]:
mrev_backtest[0].tail(25)

date
2023-11-29   -0.011616
2023-11-30   -0.011677
2023-12-01   -0.011683
2023-12-04   -0.009052
2023-12-05   -0.010295
2023-12-06   -0.012279
2023-12-07   -0.009298
2023-12-08   -0.012947
2023-12-11   -0.014303
2023-12-12   -0.008456
2023-12-13   -0.012293
2023-12-14   -0.013086
2023-12-15   -0.010704
2023-12-18   -0.010065
2023-12-19   -0.011753
2023-12-20   -0.011061
2023-12-21   -0.010825
2023-12-22   -0.010694
2023-12-26   -0.008767
2023-12-27   -0.007877
2023-12-28   -0.008688
2023-12-29   -0.008558
2024-01-02   -0.011946
2024-01-03   -0.010501
2024-01-04   -0.012461
dtype: float64

In [95]:
mrev_livetest[0].tail(25)

date
2023-11-29   -0.011616
2023-11-30   -0.011677
2023-12-01   -0.011657
2023-12-04   -0.009076
2023-12-05   -0.010275
2023-12-06   -0.012271
2023-12-07   -0.009164
2023-12-08   -0.012771
2023-12-11   -0.014302
2023-12-12   -0.008394
2023-12-13   -0.012293
2023-12-14   -0.013086
2023-12-15   -0.010704
2023-12-18   -0.010065
2023-12-19   -0.011753
2023-12-20   -0.011061
2023-12-21   -0.010825
2023-12-22   -0.010694
2023-12-26   -0.008767
2023-12-27   -0.007877
2023-12-28   -0.008452
2023-12-29   -0.008215
2024-01-02   -0.011946
2024-01-03   -0.010501
2024-01-04   -0.012461
dtype: float64

\/

# Factor Trend

In [119]:
import talib
import quantstats as qs

from fredapi import Fred
from core.operation import *
from class_trend.trend_helper import TrendHelper
from class_strat.strat import Strategy

class StratTrendMLS(Strategy):
    def __init__(self,
                 allocate=None,
                 current_date=None,
                 start_date=None,
                 threshold=None,
                 num_stocks=None,
                 window_hedge=None,
                 window_port=None):

        '''
        allocate (float): Percentage of capital to allocate for this strategy
        current_date (str: YYYY-MM-DD): Current date (this will be used as the end date for backtest period)
        start_date (str: YYYY-MM-DD): Start date for backtest period
        num_stocks (int): Number of stocks to long
        threshold (int): Market cap threshold to determine if a stock is buyable/shortable
        window_hedge (int): Rolling window size to calculate inverse volatility for hedge portfolio
        window_port (int): Rolling window size to calculate inverse volatility for trend portfolio
        '''

        super().__init__(allocate, current_date, threshold)
        self.allocate = allocate
        self.current_date = current_date
        self.start_date = start_date
        self.threshold = threshold
        self.num_stocks = num_stocks
        self.window_hedge = window_hedge
        self.window_port = window_port

        with open(get_config() / 'api_key.json') as f:
            config = json.load(f)
            fred_key = config['fred_key']

        self.fred_key = fred_key

    def exec_backtest(self):
        print("-----------------------------------------------------------------BACKTEST TREND MLS-------------------------------------------------------------------------------------")
        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # -----------------------------------------------------------------------------DATA--------------------------------------------------------------------------------------------
        # Create Trend Helper Class
        trend_helper = TrendHelper(current_date=self.current_date, start_date=self.start_date, num_stocks=self.num_stocks)

        # Params
        live = True

        # Load in datasets
        historical_price = pd.read_parquet(get_parquet(live) / 'data_price.parquet.brotli')
        market = pd.read_parquet(get_parquet(live) / 'data_misc.parquet.brotli', columns=['market_cap'])
        historical_price = set_timeframe(historical_price, self.start_date, self.current_date)

        # Create returns and resample fund_q date index to daily
        price = create_return(historical_price, [1])
        price = price.merge(market, left_index=True, right_index=True, how='left')

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # --------------------------------------------------------------------CREATE BOND+COMMODITY PORT-------------------------------------------------------------------------------
        print("---------------------------------------------------------------CREATE BOND+COMMODITY PORT-------------------------------------------------------------------------------")
        # Commodities
        com_ticker = ['GLD', 'SLV', 'PDBC', 'USO', 'AMLP', 'XOP']
        com = trend_helper._get_ret(com_ticker)

        # Bonds
        bond_ticker = ['BND', 'AGG', 'BNDX', 'VCIT', 'MUB', 'VCSH', 'BSV', 'VTEB', 'IEF', 'MBB', 'GOVT', 'VGSH', 'IUSB', 'TIP']
        bond = trend_helper._get_ret(bond_ticker)

        # Create portfolio
        bond_com_port = pd.concat([bond, com], axis=0)
        bond_com_port['vol'] = bond_com_port.groupby('ticker')['RET_01'].rolling(self.window_hedge).std().reset_index(level=0, drop=True)
        bond_com_port['inv_vol'] = 1 / bond_com_port['vol']
        bond_com_port['norm_inv_vol'] = bond_com_port.groupby('date')['inv_vol'].apply(lambda x: x / x.sum()).reset_index(level=0, drop=True)
        bond_com_port['weighted_ret'] = bond_com_port['RET_01'] * bond_com_port['norm_inv_vol']
        bond_com_port = bond_com_port.groupby('date')['weighted_ret'].sum()
        bond_com_port = bond_com_port.to_frame()
        bond_com_port.columns = ['bond_comm_ret']

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # --------------------------------------------------------------------------GET MACRO DATA-------------------------------------------------------------------------------------
        print("---------------------------------------------------------------------GET MACRO DATA-------------------------------------------------------------------------------------")
        # Date Index
        date_index = bond_com_port.index
        date_index = date_index.to_frame().drop('date', axis=1).reset_index()

        # 5-Year Inflation Rate
        fred = Fred(api_key=self.fred_key)
        inflation = fred.get_series("T5YIE").to_frame()
        inflation.columns = ['5YIF']
        inflation = inflation.shift(1)
        inflation = inflation.reset_index()
        inflation = pd.merge_asof(date_index, inflation, left_on='date', right_on='index', direction='backward')
        inflation = inflation.set_index('date').drop('index', axis=1)
        inflation = inflation.ffill()

        # Unemployment Rate
        fred = Fred(api_key=self.fred_key)
        unemploy = fred.get_series("UNRATE").to_frame()
        unemploy.columns = ['UR']
        unemploy = unemploy.shift(1)
        unemploy = unemploy.reset_index()
        unemploy = pd.merge_asof(date_index, unemploy, left_on='date', right_on='index', direction='backward')
        unemploy = unemploy.set_index('date').drop('index', axis=1)
        unemploy = unemploy.ffill()

        # 10-year vs. 2-year Yield Curve
        fred = Fred(api_key=self.fred_key)
        yield_curve = fred.get_series("T10Y2Y").to_frame()
        yield_curve.columns = ['YIELD']
        yield_curve = yield_curve.shift(1)
        yield_curve = yield_curve.reset_index()
        yield_curve = pd.merge_asof(date_index, yield_curve, left_on='date', right_on='index', direction='backward')
        yield_curve = yield_curve.set_index('date').drop('index', axis=1)
        yield_curve = yield_curve.ffill()

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # --------------------------------------------------------------------------CREATE SIGNALS-------------------------------------------------------------------------------------
        print("---------------------------------------------------------------------CREATE SIGNALS-------------------------------------------------------------------------------------")
        # Exponential Moving Averages
        for t in [60, 252]:
            price[f'ema_{t}'] = (price.groupby('permno', group_keys=False).apply(lambda x: talib.EMA(x.Close, timeperiod=t)))

        ema_buy = (price['ema_60'] > price['ema_252'])

        # Relative Strength Index
        for t in [5]:
            price[f'rsi_{t}'] = (price.groupby('permno', group_keys=False).apply(lambda x: talib.RSI(x.Close, timeperiod=t)))

        rsi_buy = (price['rsi_5'] < 30)

        # Macro Trend
        macro = pd.concat([inflation, unemploy, yield_curve], axis=1)
        macro['5YIF_z'] = (macro['5YIF'] - macro['5YIF'].mean()) / macro['5YIF'].std()
        macro['UR_z'] = (macro['UR'] - macro['UR'].mean()) / macro['UR'].std()
        macro['YIELD_z'] = (macro['YIELD'] - macro['YIELD'].mean()) / macro['YIELD'].std()
        macro['mt'] = macro[['5YIF_z', 'UR_z', 'YIELD_z']].mean(axis=1)

        for t in [21, 60]:
            macro[f'mt_{t}'] = macro['mt'].rolling(t).mean()

        macro_buy = (macro['mt_21'] > macro['mt_60'])
        macro_buy_df = macro_buy.to_frame()
        macro_buy_df.columns = ['macro_buy']

        # Market Threshold
        market_buy = (price['market_cap'] > self.threshold)

        # Volume Trend
        volume_buy = (price['Volume'] > price['Volume'].rolling(window=60).mean())

        # Create signal column
        price['signal'] = 0
        price.loc[ema_buy & volume_buy & rsi_buy & market_buy, 'signal'] = 1
        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # -----------------------------------------------------------------------------BACKTEST----------------------------------------------------------------------------------------
        print("------------------------------------------------------------------------BACKTEST----------------------------------------------------------------------------------------")
        # Create trend portfolio
        price['vol'] = price.groupby('permno')['RET_01'].rolling(self.window_port).std().reset_index(level=0, drop=True)
        price['inv_vol'] = 1 / price['vol']
        trend_port = price.groupby('date').apply(trend_helper._top_inv_vol).reset_index(level=0, drop=True)
        trend_port['norm_inv_vol'] = trend_port.groupby('date')['inv_vol'].apply(lambda x: x / x.sum()).reset_index(level=0, drop=True)
        trend_port['weighted_ret'] = trend_port['RET_01'] * trend_port['norm_inv_vol'] * trend_port['signal']
        trend_ret = trend_port.groupby('date')['weighted_ret'].sum()
        trend_ret = trend_ret.to_frame()
        trend_ret.columns = ['inv_vol_ret']

        # Combine trend portfolio + hedge portfolio
        total_ret = pd.merge(trend_ret, bond_com_port, left_index=True, right_index=True, how='left')
        total_ret = total_ret.merge(macro_buy_df, left_index=True, right_index=True, how='left')
        col1, col2 = total_ret.columns[0], total_ret.columns[1]
        total_ret['total_ret'] = total_ret.apply(trend_helper._calc_total_port, args=(col1, col2), axis=1)
        total_daily_ret = total_ret['total_ret']

        return trend_port, macro_buy_df

    def exec_live(self):
        print("-------------------------------------------------------------------EXEC TREND MLS---------------------------------------------------------------------------------------")
        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # -----------------------------------------------------------------------------DATA--------------------------------------------------------------------------------------------
        # Create Trend Helper Class
        trend_helper = TrendHelper(current_date=self.current_date, start_date=self.start_date, num_stocks=self.num_stocks)

        # Params
        live = True

        # Load in datasets
        price = pd.read_parquet(get_parquet(live) / 'data_price.parquet.brotli')
        market = pd.read_parquet(get_parquet(live) / 'data_misc.parquet.brotli', columns=['market_cap'])
        price = set_timeframe(price, self.start_date, self.current_date)

        # Create returns
        # Commodities
        com_ticker = ['GLD', 'SLV', 'PDBC', 'USO', 'AMLP', 'XOP']
        com = trend_helper._get_ret(com_ticker)
        # Bonds
        bond_ticker = ['BND', 'AGG', 'BNDX', 'VCIT', 'MUB', 'VCSH', 'BSV', 'VTEB', 'IEF', 'MBB', 'GOVT', 'VGSH', 'IUSB', 'TIP']
        bond = trend_helper._get_ret(bond_ticker)
        price = create_return(price, [1])
        price = price.merge(market, left_index=True, right_index=True, how='left')

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # --------------------------------------------------------------------CREATE BOND+COMMODITY PORT-------------------------------------------------------------------------------
        print("---------------------------------------------------------------CREATE BOND+COMMODITY PORT-------------------------------------------------------------------------------")
        # Create portfolio
        bond_com_port = pd.concat([bond, com], axis=0)
        window_bond_com_port = window_data(data=bond_com_port, date=self.current_date, window=self.window_hedge*2)
        window_bond_com_port['vol'] = window_bond_com_port.groupby('ticker')['RET_01'].rolling(self.window_hedge).std().reset_index(level=0, drop=True)
        window_bond_com_port['inv_vol'] = 1 / window_bond_com_port['vol']
        window_bond_com_port['weight'] = window_bond_com_port.groupby('date')['inv_vol'].apply(lambda x: x / x.sum()).reset_index(level=0, drop=True)

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # --------------------------------------------------------------------------GET MACRO DATA-------------------------------------------------------------------------------------
        print("---------------------------------------------------------------------GET MACRO DATA-------------------------------------------------------------------------------------")
        # Date Index
        date_index = window_bond_com_port.index.get_level_values('date').unique()
        date_index = date_index.to_frame().drop('date', axis=1).reset_index()
        
        # 5-Year Inflation Rate
        fred = Fred(api_key=self.fred_key)
        inflation = fred.get_series("T5YIE").to_frame()
        inflation.columns = ['5YIF']
        inflation = inflation.shift(1)
        inflation = inflation.reset_index()
        inflation = pd.merge_asof(date_index, inflation, left_on='date', right_on='index', direction='backward')
        inflation = inflation.set_index('date').drop('index', axis=1)
        inflation = inflation.ffill()

        # Unemployment Rate
        fred = Fred(api_key=self.fred_key)
        unemploy = fred.get_series("UNRATE").to_frame()
        unemploy.columns = ['UR']
        unemploy = unemploy.shift(1)
        unemploy = unemploy.reset_index()
        unemploy = pd.merge_asof(date_index, unemploy, left_on='date', right_on='index', direction='backward')
        unemploy = unemploy.set_index('date').drop('index', axis=1)
        unemploy = unemploy.ffill()

        # 10-year vs. 2-year Yield Curve
        fred = Fred(api_key=self.fred_key)
        yield_curve = fred.get_series("T10Y2Y").to_frame()
        yield_curve.columns = ['YIELD']
        yield_curve = yield_curve.shift(1)
        yield_curve = yield_curve.reset_index()
        yield_curve = pd.merge_asof(date_index, yield_curve, left_on='date', right_on='index', direction='backward')
        yield_curve = yield_curve.set_index('date').drop('index', axis=1)
        yield_curve = yield_curve.ffill()

        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # --------------------------------------------------------------------------CREATE SIGNALS-------------------------------------------------------------------------------------
        print("---------------------------------------------------------------------CREATE SIGNALS-------------------------------------------------------------------------------------")
        # Window Data
        window_price = window_data(data=price, date=self.current_date, window=252*2)

        # Exponential Moving Averages
        for t in [60, 252]:
            window_price[f'ema_{t}'] = (window_price.groupby('permno', group_keys=False).apply(lambda x: talib.EMA(x.Close, timeperiod=t)))

        ema_buy = (window_price['ema_60'] > window_price['ema_252'])

        # Relative Strength Index
        for t in [5]:
            window_price[f'rsi_{t}'] = (window_price.groupby('permno', group_keys=False).apply(lambda x: talib.RSI(x.Close, timeperiod=t)))

        rsi_buy = (window_price['rsi_5'] < 30)

        # Macro Trend
        macro = pd.concat([inflation, unemploy, yield_curve], axis=1)
        window_macro = macro.tail(60*2)
        window_macro['5YIF_z'] = (window_macro['5YIF'] - window_macro['5YIF'].mean()) / window_macro['5YIF'].std()
        window_macro['UR_z'] = (window_macro['UR'] - window_macro['UR'].mean()) / window_macro['UR'].std()
        window_macro['YIELD_z'] = (window_macro['YIELD'] - window_macro['YIELD'].mean()) / window_macro['YIELD'].std()
        window_macro['mt'] = window_macro[['5YIF_z', 'UR_z', 'YIELD_z']].mean(axis=1)

        for t in [21, 60]:
            window_macro[f'mt_{t}'] = window_macro['mt'].rolling(t).mean()

        macro_buy = (window_macro['mt_21'] > window_macro['mt_60'])
        macro_buy_df = macro_buy.to_frame()
        macro_buy_df.columns = ['macro_buy']

        # Market Threshold
        market_buy = (window_price['market_cap'] > self.threshold)

        # Volume Trend
        volume_buy = (window_price['Volume'] > window_price['Volume'].rolling(window=60).mean())

        # Create signal column
        window_price['signal'] = 0
        window_price.loc[ema_buy & volume_buy & rsi_buy & market_buy, 'signal'] = 1
        # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # -----------------------------------------------------------------------------GET STOCKS--------------------------------------------------------------------------------------
        print("------------------------------------------------------------------------GET STOCKS--------------------------------------------------------------------------------------")
        # Create trend portfolio
        window_price['vol'] = window_price.groupby('permno')['RET_01'].rolling(self.window_port).std().reset_index(level=0, drop=True)
        window_price['inv_vol'] = 1 / window_price['vol']
        trend_port = window_price.groupby('date').apply(trend_helper._top_inv_vol).reset_index(level=0, drop=True)
        trend_port['weight'] = trend_port.groupby('date')['inv_vol'].apply(lambda x: x / x.sum()).reset_index(level=0, drop=True)

        # Total portfolio allocation weights
        macro_buy_df = macro_buy_df.loc[macro_buy_df.index.get_level_values('date') == self.current_date]
        if macro_buy_df.values[0]:
            trend_factor = 0.5
            hedge_factor = 0.5
        else:
            trend_factor = 0.25
            hedge_factor = 0.75

        return trend_port, macro_buy_df

/

In [120]:
# Initalize Factor Strategy
strat_crit = json.load(open(get_config() / 'strat_crit.json'))
current_date = '2024-01-04'
strat_trend_mls = StratTrendMLS(allocate=strat_crit['trend_mls']['allocate'], current_date=current_date, start_date=strat_crit['trend_mls']['start_backtest'], threshold=strat_crit['trend_mls']['threshold'], num_stocks=strat_crit['trend_mls']['per_side'][0], window_hedge=60, window_port=252)

\

In [121]:
# Get Backtest Weights
trend_backtest = strat_trend_mls.exec_backtest()

-----------------------------------------------------------------BACKTEST TREND MLS-------------------------------------------------------------------------------------
/|---------------------------------------------------------------CREATE BOND+COMMODITY PORT-------------------------------------------------------------------------------


Fetching data:   0%|                                                                                                                                                                    | 0/6 [00:00<?, ?ticker/s]

/|

Fetching data:  17%|██████████████████████████                                                                                                                                  | 1/6 [00:00<00:04,  1.07ticker/s]

/|

...

Fetching data:  93%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████           | 13/14 [00:11<00:00,  1.22ticker/s]

|/

Fetching data: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:12<00:00,  1.14ticker/s]

-/

-\---------------------------------------------------------------------GET MACRO DATA-------------------------------------------------------------------------------------
|/---------------------------------------------------------------------CREATE SIGNALS-------------------------------------------------------------------------------------
\------------------------------------------------------------------------BACKTEST----------------------------------------------------------------------------------------
|\

In [113]:
# Get Livetest Weights
trend_livetest = strat_trend_mls.exec_live()

\-------------------------------------------------------------------EXEC TREND MLS---------------------------------------------------------------------------------------
-

Fetching data:   0%|                                                                                                                                                                    | 0/6 [00:00<?, ?ticker/s]

/

Fetching data:  17%|██████████████████████████                                                                                                                                  | 1/6 [00:01<00:06,  1.38s/ticker]

/

...

Fetching data:  93%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████           | 13/14 [00:10<00:00,  1.26ticker/s]

/

Fetching data: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:11<00:00,  1.22ticker/s]

-

----------------------------------------------------------------CREATE BOND+COMMODITY PORT-------------------------------------------------------------------------------
\---------------------------------------------------------------------GET MACRO DATA-------------------------------------------------------------------------------------
|\---------------------------------------------------------------------CREATE SIGNALS-------------------------------------------------------------------------------------
/------------------------------------------------------------------------GET STOCKS--------------------------------------------------------------------------------------
|

In [122]:
trend_backtest[0].tail(25)

,,Open,High,Low,Close,Volume,RET_01,market_cap,ema_60,ema_252,rsi_5,signal,vol,inv_vol,norm_inv_vol,weighted_ret
permno,date,,,,,,,,,,,,,,,
87432,2024-01-04,130.550,131.4950,130.1900,131.00,2446587.0,-0.001220,3.838300e+10,125.400481,125.263813,14.493734,1,0.017153,58.299090,0.023166,-0.000028
60628,2024-01-04,247.250,247.7400,245.0800,245.29,2911784.0,-0.009490,6.166860e+10,256.119404,240.024252,21.549205,1,0.017280,57.871418,0.022996,-0.000218
34032,2024-01-04,63.600,64.8989,63.6000,64.23,2584101.0,-0.000622,1.441970e+10,61.185480,56.010574,13.751978,1,0.017395,57.488112,0.022843,-0.000014
77129,2024-01-04,20.690,20.9800,20.6500,20.82,6084652.0,0.004826,1.290580e+10,19.715801,19.235355,26.245827,1,0.017769,56.277750,0.022362,0.000108
15053,2024-01-04,32.540,32.8900,32.4100,32.71,2755113.0,0.000000,7.199471e+09,31.330869,29.467082,18.861110,1,0.017841,56.050082,0.022272,0.000000
89138,2024-01-04,100.750,100.7500,98.4650,98.95,1668753.0,-0.013164,1.448311e+10,104.457658,101.673386,19.626023,1,0.017966,55.661784,0.022118,-0.000291
62498,2024-01-04,336.610,342.5500,336.6100,340.32,503578.0,0.006060,2.518368e+10,354.199294,348.867279,24.385739,1,0.018394,54.366939,0.021603,0.000131
86072,2024-01-04,267.920,274.4700,260.5100,268.40,780176.0,-0.001265,2.209657e+10,266.362851,243.489970,18.332771,1,0.018513,54.015459,0.021464,-0.000027
78223,2024-01-04,52.520,52.8550,51.9600,52.58,3452983.0,0.000381,2.608305e+10,51.457154,50.261065,25.391484,1,0.019039,52.524686,0.020871,0.000008


In [123]:
trend_livetest[0].tail(25)

,,Open,High,Low,Close,Volume,RET_01,market_cap,ema_60,ema_252,rsi_5,signal,vol,inv_vol,weight
permno,date,,,,,,,,,,,,,,
87432,2024-01-04,130.550,131.4950,130.1900,131.00,2446587.0,-0.001220,3.838300e+10,125.400481,124.754913,14.493734,1,0.017153,58.299090,0.023166
60628,2024-01-04,247.250,247.7400,245.0800,245.29,2911784.0,-0.009490,6.166860e+10,256.119406,240.717732,21.549205,1,0.017280,57.871418,0.022996
34032,2024-01-04,63.600,64.8989,63.6000,64.23,2584101.0,-0.000622,1.441970e+10,61.185480,56.120264,13.751978,1,0.017395,57.488112,0.022843
77129,2024-01-04,20.690,20.9800,20.6500,20.82,6084652.0,0.004826,1.290580e+10,19.715801,19.337804,26.245827,1,0.017769,56.277750,0.022362
15053,2024-01-04,32.540,32.8900,32.4100,32.71,2755113.0,0.000000,7.199471e+09,31.330870,29.442765,18.861110,1,0.017841,56.050082,0.022272
89138,2024-01-04,100.750,100.7500,98.4650,98.95,1668753.0,-0.013164,1.448311e+10,104.457657,102.312695,19.626023,1,0.017966,55.661784,0.022118
62498,2024-01-04,336.610,342.5500,336.6100,340.32,503578.0,0.006060,2.518368e+10,354.199291,351.233062,24.385739,1,0.018394,54.366939,0.021603
86072,2024-01-04,267.920,274.4700,260.5100,268.40,780176.0,-0.001265,2.209657e+10,266.362853,244.292089,18.332771,1,0.018513,54.015459,0.021464
78223,2024-01-04,52.520,52.8550,51.9600,52.58,3452983.0,0.000381,2.608305e+10,51.457154,50.285881,25.391484,1,0.019039,52.524686,0.020871


|-